In [ ]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = '/home/data/train.json'

# Read the JSON file
train_data = []
with open(train_path, 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

train_df = pd.DataFrame(train_data)
print(f"Training data shape: {train_df.shape}")
print(f"Columns: {list(train_df.columns)}")